# Notebook plots for paper presented at conference EEEIC 2023

This notebook contains the code used to plot the images shown at conference EEEIC 2023.
If you use this code, please cite "D. Fioriti, T. Ferrucci, D. Poli, Fairness and reward in Energy Communities: game-theory versus simplified approaches, EEEIC 2023"

In [1]:
using Pkg
Pkg.activate(".")

  Activating new project at `c:\Users\Davide\git\gitdf\EnergyCommunity.jl\examples`


In [2]:
parent_dir = "results_final_15_3_2023/results_paper_NC"
saveenum_dir = "$parent_dir/enum"

folder_imgs = "$parent_dir/imgs_EEEIC"

n_base_users=10
add_EC=true

id_filter = [7, 11, 14, 15]

fig_size = (600, 250)

(600, 250)

In [3]:
using EnergyCommunity
using FileIO
using HiGHS, Plots, StatsPlots, CategoricalArrays
using JuMP
using Gurobi
using TheoryOfGames
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML
using CSV

mkpath(folder_imgs)

fontsize = 1
fontname = "times"

gr()

# default(
#     titlefontsize=20,
#     # tickfontsize=fontsize-2,
#     guidefontsize=fontsize-2,
#     legend_title_font_pointsize=fontsize-1,
#     labelfontsize=fontsize-2,
#     legendfontsize=fontsize-2,
# )

ArgumentError: ArgumentError: Package EnergyCommunity not found in current path.
- Run `import Pkg; Pkg.add("EnergyCommunity")` to install the EnergyCommunity package.

In [4]:
f_userlist(add_EC, n_users) = [add_EC ? [EC_CODE] : String[]; ["user$u" for u=1:n_users]]

f_userlist (generic function with 1 method)

## Store the data of the simulations

#### Enumerative configurations

In [5]:
EC_size_list_enum = []
dict_enums = Dict()

for filename in readdir(saveenum_dir)
    if endswith(filename, ".jld2") && startswith(filename, "enum_simulations_results_")
        size_enum = parse(Int, replace(filename, "enum_simulations_results_"=>"", ".jld2"=>""))
        push!(EC_size_list_enum, size_enum)
        dict_enums[size_enum] = load("$saveenum_dir/enum_simulations_results_$size_enum.jld2")
        dict_enums[size_enum]["ECmodel"] = load!("$saveenum_dir/ec_model_$size_enum.jld2", ModelEC())
        dict_enums[size_enum]["NCmodel"] = load!("$saveenum_dir/nc_model_$size_enum.jld2", ModelEC())
    end
end

UndefVarError: UndefVarError: load not defined

## Benefit distribution versus size

### Store data of reward distribution

In [6]:
df_benefit = dict_enums[10]["df_reward_enum"]
ecm = dict_enums[10]["ECmodel"]
ncm = dict_enums[10]["NCmodel"]
df_time = dict_enums[10]["df_time_enum"]

nc_obj_by_user = objective_by_user(ncm)
ec_obj_by_user = objective_by_user(ecm)

# rCO = split_financial_terms(ECModel, profit_distribution)
# df_CO = financial_terms_to_df(rCO)

# rNC = split_financial_terms(NC_Model, obj_by_user)
# df_NC ) financial_terms_to_df(rNC)

KeyError: KeyError: key 10 not found

### utils for benefit computations

In [7]:
# function to automatically create matrix blocks for aggregation on the data
function prepare_grouped(df, u_list, cols, col_user=:user_set)

    idx_list = [findfirst(u .== df[!, col_user]) for u in u_list]

    grp_x = repeat(string.(cols), inner=length(u_list))
    grp_lbls = repeat(string.(u_list), outer=length(cols))
    df_list = []
    for c=cols
        append!(df_list, df[idx_list, c])
    end

    return (
        data=Vector{Float64}(df_list),
        grp_x=grp_x,
        grp_lbls=grp_lbls,
    )
end

function filter_reward_labels!(ulist)
    return replace!(
        ulist,
        "shapley_enum"=>"Shapley",
        "nucleolus_enum"=>"Nucleolus",
        "varleastcore_enum"=>"VarLeastCore",
    )
end

function filter_user_labels!(ulist)
    return replace!(
        ulist,
        "EC"=>"Agg",
    )
end

financial_terms_to_df = r->DataFrame([:user_set=>axes(r.NPV)[1]; [k=>r[k].data for k in keys(r)]])


#13 (generic function with 1 method)

### Show benefit (delta NPV)

In [8]:
cols = ["shapley_enum", "nucleolus_enum", "varleastcore_enum"]
u_list = f_userlist(true, 10)
out = prepare_grouped(
    df_benefit,
    u_list,
    cols
)

filter_reward_labels!(out.grp_x)
filter_user_labels!(out.grp_lbls)
filter_user_labels!(u_list)

p = groupedbar(
        CategoricalArray(out.grp_lbls, levels = u_list),
        out.data ./ 1000,
        group=out.grp_x,
        framestyle=:box,
        legendtitle="Reward scheme",
        xlabel="User/Agg",
        ylabel="Net benefit (DNPV) [k€]",
        # title=title_plot[k],
        # yerror=(min_err, max_err),
        # legend=:outerright,
        # legend=:right,
        ylims=[0, 120],
        # yaxis=:log,
        size=fig_size,
    )
display(p)

savefig(p, "$folder_imgs/benefit.pdf");

UndefVarError: UndefVarError: EC_CODE not defined

### Plot shared energy

In [9]:
per_unit = false
only_shared = true

shared_cons = calculate_shared_consumption(ecm; per_unit=per_unit, only_shared=only_shared)
shared_prod = calculate_shared_production(ecm; per_unit=per_unit, only_shared=only_shared)
shared_tot = shared_cons .+ shared_prod

self_cons = calculate_self_consumption(ecm; per_unit=per_unit)
self_prod = calculate_self_production(ecm; per_unit=per_unit)

data_list_content = [shared_cons, shared_prod, shared_tot, self_cons] #, self_prod]
data_list_tags = ["Shared cons.", "Shared prod.", "Shared cons. + prod.", "Self cons./prod."]#, "Self prod."]

u_list = f_userlist(false, 10)

data_y = Float64[]
data_x = []
data_group = []
for (dd, lbl) in zip(
        data_list_content,
        data_list_tags,
    )

    append!(data_x, u_list)
    append!(data_y, dd[u_list].data)
    append!(data_group, repeat([lbl], length(dd[u_list].data)))
end

out = (
    data=data_y,
    grp_x=data_x,
    grp_lbls=data_group,
)


filter_reward_labels!(out.grp_x)
filter_user_labels!(out.grp_lbls)


p = groupedbar(
        CategoricalArray(out.grp_x, levels = u_list),
        out.data ./1000,
        group=CategoricalArray(out.grp_lbls, levels = data_list_tags),
        framestyle=:box,
        # legendtitle="Reward scheme",
        xlabel="User/Agg",
        ylabel="[MWh]",
        # title=title_plot[k],
        # yerror=(min_err, max_err),
        # legend=:outerright,
        # legend=:right,
        # ylims=[1, 300],
        # yaxis=:log,
        size=fig_size,
    )

display(p)

savefig(p, "$folder_imgs/shared_self_energy.pdf");

UndefVarError: UndefVarError: calculate_shared_consumption not defined

### Calculate main financial-related terms

#### Prepare financial terms

In [10]:
# Calculate financial dictionary of elements
dict_financial = Dict{String, Any}()

for c in cols

    rew = JuMP.Containers.DenseAxisArray(df_benefit[!, c], df_benefit[!, :user_set])

    profit_distribution = JuMP.Containers.DenseAxisArray(
        collect([rew[k] + nc_obj_by_user[k] for k in df_benefit[!, :user_set]]),
        df_benefit[:, :user_set],
    )

    rCO = split_financial_terms(ecm, profit_distribution)

    dict_financial[c] = rCO

end

rNC = split_financial_terms(ncm, nc_obj_by_user)
rCO = split_financial_terms(ecm, nc_obj_by_user);

UndefVarError: UndefVarError: JuMP not defined

#### Main costs EC vs NC

In [11]:
df_CAPEX_OEM = df_benefit[:, [:user_set]]

df_CAPEX_OEM[!, "CAPEX - CO"] = [rCO.CAPEX[k] for k in df_CAPEX_OEM[!, :user_set]]
df_CAPEX_OEM[!, "OPEX - CO"] = [rCO.OPEX[k] for k in df_CAPEX_OEM[!, :user_set]]
df_CAPEX_OEM[!, "CAPEX - NC"] = [rNC.CAPEX[k] for k in df_CAPEX_OEM[!, :user_set]]
df_CAPEX_OEM[!, "OPEX - NC"] = [rNC.OPEX[k] for k in df_CAPEX_OEM[!, :user_set]]

cols = names(df_CAPEX_OEM)[2:end]

u_list = f_userlist(true, 10)
out = prepare_grouped(
    df_CAPEX_OEM,
    u_list,
    cols
)

# filter_reward_labels!(out.grp_x)
filter_user_labels!(out.grp_lbls)
filter_user_labels!(u_list)
high = .9
low = .1
mid = .4
colors_grouped = [RGB(high,low,low),RGB(high,mid,mid),RGB(low,low,high), RGB(mid,mid,high)]

p = groupedbar(
        CategoricalArray(out.grp_lbls, levels = u_list),
        out.data ./ 1000,
        group=out.grp_x,
        framestyle=:box,
        legendtitle="Reward scheme",
        xlabel="User",
        ylabel="[k€]",
        # title=title_plot[k],
        # yerror=(min_err, max_err),
        # legend=:outerright,
        # legend=:right,
        # ylims=[1, 300],
        # yaxis=:log,
        color = reshape(colors_grouped, (1,4)),
        size=fig_size,
    )

display(p)

savefig(p, "$folder_imgs/capex_opex.pdf");

UndefVarError: UndefVarError: df_benefit not defined

#### Reward allocation

In [12]:
cols = ["shapley_enum", "nucleolus_enum", "varleastcore_enum"]

df_reward = df_benefit[:, [:user_set]]

for c in cols
    r = dict_financial[c]
    df_reward[!, c] = [r.REWARD[k] for k in df_benefit[!, :user_set]]
end

u_list = f_userlist(true, 10)
out = prepare_grouped(
    df_reward,
    u_list,
    cols
)

filter_reward_labels!(out.grp_x)
filter_user_labels!(out.grp_lbls)
filter_user_labels!(u_list)

p = groupedbar(
        CategoricalArray(out.grp_lbls, levels = u_list),
        out.data ./ 1000,
        group=out.grp_x,
        framestyle=:box,
        legendtitle="Reward scheme",
        xlabel="User/Agg",
        ylabel="Reward allocation [k€]",
        # title=title_plot[k],
        # yerror=(min_err, max_err),
        # legend=:outerright,
        # legend=:right,
        ylims=[1, 120],
        # yaxis=:log,
        size=fig_size,
    )

display(p)

savefig(p, "$folder_imgs/reward.pdf");

UndefVarError: UndefVarError: df_benefit not defined

#### Simplified schemes plots

In [13]:
df_simplified = df_reward[:, [:user_set]]

shared_tot_vec = [ k == "EC" ? 0.0 : shared_tot[k] for k in df_simplified[!, :user_set]]
capex_vec = [rCO.CAPEX[k] for k in df_simplified[!, :user_set]]

ann_factor = sum(Float64[1/(1+field(ecm.gen_data, "d_rate"))^y for y = 1:field(ecm.gen_data, "project_lifetime")])

df_simplified[!, "Shared scheme [€/kWh]"] = df_reward[!, "varleastcore_enum"] ./ shared_tot_vec / ann_factor
df_simplified[!, "Financial scheme [€/€]"] = df_reward[!, "varleastcore_enum"] ./ capex_vec

u_list = f_userlist(true, 10)
out = prepare_grouped(
    df_simplified,
    u_list,
    names(df_simplified)[2:end],
)

# filter_reward_labels!(out.grp_x)
filter_user_labels!(out.grp_lbls)
filter_user_labels!(u_list)

p = groupedbar(
        CategoricalArray(out.grp_lbls, levels = u_list),
        out.data,
        group=out.grp_x,
        framestyle=:box,
        legendtitle="Reward scheme",
        xlabel="User/Agg",
        ylabel="Eq. price",
        # title=title_plot[k],
        # yerror=(min_err, max_err),
        # legend=:outerright,
        # legend=:right,
        ylims=[0, 0.35],
        # yaxis=:log,
        size=fig_size,
    )

display(p)

savefig(p, "$folder_imgs/simplified_allocations.pdf");

UndefVarError: UndefVarError: df_reward not defined

In [14]:
df_simplified